In [17]:
import json
import pandas
import numpy as np
import plotly.express as px

MapboxToken = "pk.eyJ1IjoiaGFzaGNvbG9uIiwiYSI6ImNrcXQ5MmphbzA1eHUyb21kOXR0bmFmejcifQ.xamoLzfJsLEryttWEJ0D8g"
imageSize = {
    "width": 1920,
    "height": 1080,
    "zoom": 8
}

def Convert_FeatureCollection2Df(featureCollection, tag):
    # list to collect dataframe of child features
    dflist = []
    # flag to cut down line segments
    linecnt = 0

    # loop child features
    for f in featureCollection['features']:
        # if child is a featurecollection
        if f['type'] == "FeatureCollection":
            df_tmp = Convert_FeatureCollection2Df(f, tag)
        # if child is a feature
        elif f['type'] == "Feature":
            df_tmp = Convert_Feature2Df(f, tag)

        # add current line count to childs
        df_tmp['lineno'] = df_tmp['lineno'] + linecnt
        # renew line count
        linecnt = df_tmp['lineno'].max() + 1
        # collect the results
        dflist.append(df_tmp)
    # merge collected results to a datafrmae
    df = pandas.concat(dflist, ignore_index=True)
    return df


def Convert_Feature2Df(feature, tag):
    lats = []
    lons = []
    tags = []
    line = []
    # works only form linestring type
    if feature['geometry']['type'] == "LineString":
        for pos in feature['geometry']['coordinates']:
            lons.append(pos[0])
            lats.append(pos[1])
            tags.append(tag)
            line.append(0)
    d = {'lon': lons, 'lat': lats, 'tag': tags, 'lineno': line}
    df = pandas.DataFrame(data=d)
    return df


def Convert_Geojson2Df(geoJsonData, tag):    
    if geoJsonData['type'] == "FeatureCollection":
        df = Convert_FeatureCollection2Df(geoJsonData, tag)
    elif geoJsonData['type'] == "Feature":
        df = Convert_Feature2Df(geoJsonData, tag)
    return df

def CreateImage_RouteLists(routeList, tagList, imgFilepath):
    geoJsonDfList = []
    linecnt = 0 
    for route, tag in zip(routeList, tagList):
        tmpGeoJsonFilePath = route[0]
        routeIdx = route[1]
        tmpGeoJson = open(tmpGeoJsonFilePath, 'r')
        tmpGeoJson = json.load(tmpGeoJson)
        
        tmpGeoJsonDf = Convert_Geojson2Df(tmpGeoJson, tag)        
        # fileter geojson by route index
        tmpGeoJsonDf = tmpGeoJsonDf[tmpGeoJsonDf['lineno']==routeIdx]
        # change lineno        
        tmpGeoJsonDf['lineno'] = linecnt
        linecnt += 1
        # append to geoJsonDflist
        geoJsonDfList.append(tmpGeoJsonDf)
        
    df = pandas.concat(geoJsonDfList, ignore_index=True)    
    Fig = px.line_mapbox(df, lat="lat", lon="lon",
                         color="tag", line_group='lineno', zoom=imageSize['zoom'])
    Fig.update_layout(mapbox_style="dark", mapbox_accesstoken=MapboxToken)
    Fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
    Fig.write_image(
        imgFilepath, width=imageSize['width'], height=imageSize['height'])
    return Fig

testRouteList = [    
    ["/home/cadit/WTK/FelineExp/ExpReports2/LCSS/Inputs/GeoJson/mmsi441908000_imo8705319_120.json", 0],
    ["/home/cadit/WTK/FelineExp/ExpReports2/LCSS/Inputs/GeoJson/mmsi441908000_imo8705319_50.json",  0]]

testTagList = ['1','2']
filename = './testres.jpg'

img = CreateImage_RouteLists(testRouteList, testTagList, filename)
img.show()

            lon        lat tag  lineno
0    126.540600  33.528867   1       0
1    126.558133  33.540550   1       0
2    126.576483  33.551117   1       0
3    126.594533  33.562200   1       0
4    126.612200  33.573833   1       0
..          ...        ...  ..     ...
135  129.133267  35.057200   1       0
136  129.123517  35.075833   1       0
137  129.098467  35.086733   1       0
138  129.079217  35.097600   1       0
139  129.056017  35.104900   1       0

[140 rows x 4 columns]
            lon        lat tag  lineno
0    129.040950  35.102550   2       0
1    129.064883  35.105367   2       0
2    129.083467  35.094517   2       0
3    129.103183  35.083283   2       0
4    129.117850  35.068850   2       0
..          ...        ...  ..     ...
138  126.616800  33.573450   2       0
139  126.595300  33.562167   2       0
140  126.575767  33.551433   2       0
141  126.556483  33.541050   2       0
142  126.540167  33.528267   2       0

[143 rows x 4 columns]
